---

# 📚 Latar Belakang

Indonesia dikenal sebagai salah satu negara yang terletak di wilayah **Cincin Api Pasifik (Pacific Ring of Fire)** 🌋, sebuah zona geologi aktif yang rentan terhadap aktivitas seismik seperti gempa bumi dan letusan gunung berapi. Hal ini menyebabkan **frekuensi gempa di Indonesia sangat tinggi**, mulai dari skala kecil yang sering tidak terasa, hingga gempa besar yang berdampak signifikan terhadap kehidupan masyarakat.

Namun, meskipun gempa adalah fenomena yang sering terjadi, **masyarakat dan pemangku kebijakan masih sering kesulitan untuk memahami pola dan distribusinya**. Hal ini dikarenakan data gempa yang tersebar, tidak selalu rapi, serta sulit untuk dianalisis secara langsung tanpa alat bantu visual.

---

## 🎯 Permasalahan yang Ingin Diselesaikan

🧩 Permasalahan utama yang ingin diselesaikan melalui proyek ini adalah:

- 📌 **Kurangnya pemahaman visual terhadap distribusi gempa bumi di Indonesia** dari waktu ke waktu.
- 📌 **Data yang tersedia dalam format mentah** cenderung tidak intuitif dan sulit dimaknai secara langsung.
- 📌 **Kebutuhan akan alat bantu yang interaktif dan informatif** untuk membantu menginterpretasi tren dan karakteristik gempa (seperti kedalaman, magnitudo, dan lokasi).

---

## 🛠️ Solusi yang Ditawarkan

Melalui proyek ini, kami melakukan:

- **Penggabungan dan pembersihan data gempa** dari berbagai sumber terpercaya (USGS dan Kaggle).
- **Preprocessing yang bertujuan merapikan data**, bukan memperbaiki nilai (misalnya bukan untuk imputasi missing value, melainkan seperti normalisasi lokasi, kategorisasi, dll).
- **Pembuatan visualisasi interaktif berbasis web** yang memperlihatkan distribusi gempa berdasarkan:

  - 🌍 **Lokasi geografis** (melalui pemetaan)
  - 🕒 **Waktu kejadian**
  - 📊 **Kedalaman dan magnitudo**

> Visualisasi ini akan dibangun menggunakan **[D3.js](https://d3js.org/)**, sebuah pustaka JavaScript yang powerful untuk manipulasi data berbasis DOM dan SVG, sehingga **interaktif, informatif, dan bisa diakses langsung melalui browser**.

---

Dengan pendekatan ini, kami berharap:

- Masyarakat dapat **lebih memahami sebaran gempa secara geografis dan temporal**.
- Peneliti dan pengambil kebijakan dapat menggunakan data ini sebagai **referensi dalam mitigasi risiko bencana**.
- Visualisasi yang disajikan mampu **mendekatkan informasi ilmiah kepada publik secara visual dan intuitif**.

---

# 📥 Load & Gabungkan Dataset Gempa Indonesia (Raw)

## 🌍 Sumber Dataset

Proyek ini menggunakan dua sumber data utama tentang kejadian gempa di wilayah Indonesia:

1. **USGS (United States Geological Survey)**  
   - Data diperoleh dengan proses **scraping manual menggunakan Python**
   - File scraping berada di: `script/USGS_ID.py`  
   - Dataset tersimpan sebagai: `usgs_earthquake_data_ID.csv`

2. **Kaggle Dataset**  
   - Sumber: [Earthquakes in Indonesia - Kaggle](https://www.kaggle.com/datasets/kekavigi/earthquakes-in-indonesia/data)  
   - Dataset tersimpan sebagai: `kaggle_earthquakes_ID.csv`

📁 Kedua file ini disimpan di direktori: `./collectiondata/`

---

## 🐍 Kode Python Penggabungan Dataset

In [25]:
import pandas as pd
import re

> **Import library**:  
Kita menggunakan `pandas` untuk manipulasi data dan `re` (regex) yang akan digunakan nanti dalam preprocessing tahap selanjutnya.


### 🔽 1. Load Dataset Mentah

In [ ]:
# Load dataset mentah
usgs_df = pd.read_csv("./collectiondata/usgs_earthquake_data_ID.csv")
kaggle_df = pd.read_csv("./collectiondata/kaggle_earthquakes_ID.csv")


📖 **Penjelasan**:  
Membaca dua dataset mentah dari hasil scraping dan sumber publik. Keduanya memiliki struktur kolom yang berbeda sehingga perlu disesuaikan terlebih dahulu.

### 🔧 2. Penyesuaian Nama Kolom Kaggle

In [ ]:
# Penyesuaian nama kolom kaggle agar seragam
kaggle_df = kaggle_df.rename(columns={
    "ot": "time",
    "lat": "latitude",
    "lon": "longitude",
    "depth": "depth",
    "mag": "magnitude",
    "remark": "place"
})

⚙️ **Penjelasan**:  
Kolom dari Kaggle disesuaikan dengan format yang sama seperti USGS agar proses penggabungan berjalan lancar (`rename` kolom).

### 🏷️ 3. Tambahkan Kolom Sumber Data

In [ ]:
# Tambahkan kolom sumber
usgs_df["source"] = "USGS"
kaggle_df["source"] = "Kaggle"

🔍 **Penjelasan**:  
Untuk keperluan analisis dan pelacakan data, ditambahkan kolom baru bernama `source` yang berisi informasi apakah data berasal dari **USGS** atau **Kaggle**.

### ✂️ 4. Pilih Kolom Penting

In [ ]:
# Ambil hanya kolom penting
usgs_selected = usgs_df[["time", "latitude", "longitude", "depth", "magnitude", "place", "source"]]
kaggle_selected = kaggle_df[["time", "latitude", "longitude", "depth", "magnitude", "place", "source"]]

✅ **Penjelasan**:  
Kita hanya memilih kolom yang relevan untuk analisis dan visualisasi:
- Waktu (`time`)
- Koordinat (`latitude`, `longitude`)
- Parameter seismik (`depth`, `magnitude`)
- Lokasi (`place`)
- Asal data (`source`)

### 🔗 5. Gabungkan Kedua Dataset

In [ ]:
# Gabungkan kedua dataset mentah
combined_raw = pd.concat([usgs_selected, kaggle_selected], ignore_index=True)

🧩 **Penjelasan**:  
Menggabungkan dua dataset menjadi satu DataFrame.  
Parameter `ignore_index=True` memastikan index di-reset dari awal.

### 💾 6. Simpan Dataset Gabungan Mentah

In [ ]:
# Simpan hasil gabungan awal ke file
combined_raw.to_csv("./collectiondata/combined_earthquakes_raw.csv", index=False)

📝 **Penjelasan**:  
Dataset gabungan mentah disimpan ke file `combined_earthquakes_raw.csv` yang nantinya akan diproses lebih lanjut dalam tahapan *preprocessing* berikutnya.

### 🧪 7. Cek Data Awal

In [ ]:
# Info
print("✅ Data berhasil digabungkan!")
print("Jumlah Baris:", combined_raw.shape[0])
print("Jumlah Kolom:", combined_raw.shape[1])

# Tampilkan data contoh
combined_raw.head()


🔎 **Penjelasan**:  
Menampilkan informasi jumlah baris dan kolom serta beberapa data awal untuk memverifikasi hasil penggabungan dataset.

---

# **🧹 Preprocessing Data Gempa Indonesia**

## **📌 Pendahuluan**

Proses *preprocessing* data ini **tidak difokuskan pada penanganan umum seperti missing values, outliers, atau normalisasi angka**. Sebaliknya, tahap ini lebih berfungsi untuk:

✅ **Membersihkan noise teks**  
✅ **Menstandarkan informasi lokasi**  
✅ **Mengklasifikasikan kategori tertentu (magnitudo, kedalaman, dan region)**  
✅ **Mengorganisir ulang kolom supaya siap digunakan untuk analisis atau visualisasi**

File yang digunakan adalah gabungan dua sumber (`USGS` dan `Kaggle`) yang sebelumnya telah digabung menjadi `combined_earthquakes_raw.csv`.

## 🧩 Langkah 1: Load Dataset Gabungan

In [18]:
# Load dataset gabungan mentah
combined_df = pd.read_csv("./collectiondata/combined_earthquakes_raw.csv")

📖 **Penjelasan**:  
Kita menggunakan `pandas` untuk membaca dataset mentah hasil penggabungan dua sumber. Dataset ini menjadi bahan utama untuk proses selanjutnya.

## ✂️ Langkah 2: Membersihkan Nama Lokasi (`place`)

In [19]:
# 🔧 Bersihkan kolom place agar hanya menyisakan nama lokasi utama
def clean_place_name(place):
    if pd.isna(place):
        return "Unknown"
    
    # Hilangkan arah dan jarak di depan (misal "62 km W of ")
    place = re.sub(r"^\d+\s*km\s*[A-Z]*\s*of\s*", "", place, flags=re.IGNORECASE)
    
    # Hilangkan ", Indonesia" atau "Indonesia" di akhir
    place = re.sub(r",?\s*indonesia\s*$", "", place, flags=re.IGNORECASE)
    
    return place.strip()

combined_df["place"] = combined_df["place"].apply(clean_place_name)


📖 **Penjelasan**:  
Beberapa nama lokasi memiliki noise seperti `"62 km W of Banda Aceh"` atau `"Padang, Indonesia"`.  
Blok kode ini menghapus:

- Jarak & arah (`"62 km W of"` jadi `"Banda Aceh"`)
- Akhiran `"Indonesia"` agar lebih ringkas dan bisa dikategorikan lebih akurat nanti

## 🗺️ Langkah 3: Klasifikasi Wilayah (`region`)

In [20]:
# Tambahkan kolom region berdasarkan 'place'
def extract_region(place):
    if pd.isna(place) or place.strip() == "":
        return "Indonesia"
    place = place.lower()
    if "sumatra" in place or "aceh" in place or "padang" in place:
        return "Sumatra"
    elif "java" in place or "jakarta" in place or "yogyakarta" in place or "bandung" in place:
        return "Java"
    elif "kalimantan" in place or "borneo" in place:
        return "Kalimantan"
    elif "sulawesi" in place or "makassar" in place or "palu" in place:
        return "Sulawesi"
    elif "papua" in place or "irian" in place:
        return "Papua"
    elif "bali" in place or "ntt" in place or "ntb" in place:
        return "Nusa Tenggara"
    elif "maluku" in place or "ambon" in place:
        return "Maluku"
    else:
        return "Indonesia"  # Default jika tidak spesifik

combined_df["region"] = combined_df["place"].apply(extract_region)


📖 **Penjelasan**:  
Blok ini mengkategorikan lokasi (`place`) ke dalam region geografis utama Indonesia seperti: `Sumatra`, `Java`, `Kalimantan`, dll.  
Tujuannya agar analisis bisa lebih terfokus per wilayah.


## 📊 Langkah 4: Klasifikasi Magnitudo

In [21]:
# Tambahkan kolom magnitude_category
def categorize_magnitude(mag):
    if pd.isna(mag):
        return "Unknown"
    elif mag < 2.0:
        return "Micro"
    elif mag < 4.0:
        return "Minor"
    elif mag < 5.0:
        return "Light"
    elif mag < 6.0:
        return "Moderate"
    elif mag < 7.0:
        return "Strong"
    elif mag < 8.0:
        return "Major"
    else:
        return "Great"

combined_df["magnitude_category"] = combined_df["magnitude"].apply(categorize_magnitude)


📖 **Penjelasan**:  
Magnitudo dikategorikan ke dalam level seperti:

- `Micro` (< 2.0)
- `Minor` (2.0 - 3.9)
- ...
- `Great` (≥ 8.0)

Ini penting untuk analisis dampak dan intensitas gempa.

## 🧭 Langkah 5: Klasifikasi Kedalaman

In [22]:
# Tambahkan kolom depth_category
def categorize_depth(depth):
    if pd.isna(depth):
        return "Unknown"
    elif depth < 70:
        return "Shallow"
    elif depth < 300:
        return "Intermediate"
    else:
        return "Deep"

combined_df["depth_category"] = combined_df["depth"].apply(categorize_depth)


📖**Penjelasan**:  
Depth (kedalaman) gempa dikategorikan menjadi:

- `Shallow` (< 70 km)
- `Intermediate` (70–299 km)
- `Deep` (≥ 300 km)

Kategori ini sering digunakan untuk memahami jenis gempa dan potensi kerusakannya.

## 🗃️ Langkah 6: Urutkan Kolom

In [ ]:
# Urutkan kolom sesuai permintaan
final_columns = [
    "time", "region", "place", "latitude", "longitude",
    "depth", "magnitude", "magnitude_category", "depth_category", "source"
]
combined_df = combined_df[final_columns]


📖**Penjelasan**:  
Kolom-kolom disusun ulang supaya konsisten dan mudah dibaca. Ini penting untuk *clean output* terutama saat digunakan dalam analisis selanjutnya.

## 💾 Langkah 7: Simpan Dataset yang Sudah Diproses

In [ ]:
# Simpan hasil preprocessing ke file CSV baru
combined_df.to_csv("./collectiondata/combined_earthquakes_ID.csv", index=False)

📖**Penjelasan**:  
Dataset hasil preprocessing disimpan dalam file baru `combined_earthquakes_ID.csv` — siap untuk digunakan!

## 📌 Ringkasan Output

In [24]:
# Info
print("\n✅ Dataset berhasil dipreproses dan disimpan!")
print("Jumlah Baris:", combined_df.shape[0])
print("Jumlah Kolom:", combined_df.shape[1])

# Tampilkan contoh data
print("\nContoh data:")
combined_df.head()



✅ Dataset berhasil dipreproses dan disimpan!
Jumlah Baris: 113219
Jumlah Kolom: 10

Contoh data:


,time,region,place,latitude,longitude,depth,magnitude,magnitude_category,depth_category,source
0,2025-04-04 18:07:25,Indonesia,Pariaman,-0.5348,99.5666,10.000,4.7,Light,Shallow,USGS
1,2025-04-04 14:18:07,Indonesia,Biak,-1.8587,135.7236,22.560,4.4,Light,Shallow,USGS
2,2025-04-04 13:59:51,Indonesia,Cilacap,-7.9155,109.1106,101.961,4.7,Light,Intermediate,USGS
3,2025-04-03 20:42:19,Indonesia,Gorontalo,0.5983,121.5558,95.593,4.7,Light,Intermediate,USGS
4,2025-04-03 04:03:39,Indonesia,Tobelo,2.1167,126.7590,36.700,5.9,Moderate,Shallow,USGS


📖**Penjelasan**:  
Menampilkan ringkasan jumlah data dan beberapa baris pertama sebagai preview. Ini langkah validasi kecil untuk memastikan semua berjalan dengan baik.

---

## **🧠 Kesimpulan**

Proses preprocessing ini **berfokus pada transformasi semantik dan kategorisasi** untuk mendukung:

- Analisis spasial (berdasarkan `region`)
- Analisis distribusi magnitudo & kedalaman
- Visualisasi data yang rapi dan mudah dibaca

🔧 Ini adalah tahapan penting sebelum data bisa digunakan untuk analisis lanjutan seperti visualisasi data



---

## 🧭 Skenario dan Task

### 🎯 Tujuan Umum
Visualisasi ini dirancang agar pengguna dapat:
- Mengetahui **sebaran lokasi gempa di Indonesia**
- Menganalisis **pola waktu terjadinya gempa** berdasarkan bulan dan tahun



### 📌 Skenario 1 – Warga ingin tahu daerah rawan gempa

> **Deskripsi:**  
Seorang warga yang tinggal di Indonesia ingin mengetahui daerah-daerah mana saja yang paling sering mengalami gempa bumi.

**Task:**
- Mengakses peta interaktif yang menampilkan titik-titik lokasi gempa
- Melihat detail gempa (magnitudo, kedalaman, waktu, lokasi) saat meng-hover atau klik titik
- Menggunakan filter (misalnya: tahun atau kekuatan magnitudo) untuk mempersempit data
- Menemukan pola: daerah seperti **Sumatra** atau **Papua** lebih sering mengalami gempa besar

**Visualisasi:**  
🌍 **Peta Sebaran Gempa** dengan D3.js (menggunakan geoJSON + titik lokasi dari dataset)



### 📌 Skenario 2 – Peneliti ingin melihat tren bulanan dan tahunan gempa

> **Deskripsi:**  
Seorang peneliti kebencanaan ingin mengetahui apakah terdapat pola musiman atau tahunan pada kejadian gempa bumi di Indonesia.

**Task:**
- Mengakses heatmap frekuensi gempa berdasarkan **bulan dan tahun**
- Mengarahkan kursor ke kotak tertentu untuk melihat jumlah gempa spesifik pada bulan & tahun tersebut
- Mengidentifikasi bulan atau tahun dengan frekuensi gempa tertinggi
- Menyimpulkan apakah terdapat lonjakan aktivitas gempa pada periode tertentu

**Visualisasi:**  
📊 **Heatmap Waktu** (frekuensi gempa per bulan dan tahun) dengan warna sebagai representasi intensitas

---

## ❓ Pertanyaan-Pertanyaan Penelitian

1. **Di wilayah mana saja di Indonesia gempa bumi paling sering terjadi?**  
   → Dijawab dengan **peta interaktif** yang menampilkan titik lokasi gempa secara geografis.

2. **Apakah terdapat pola tahunan atau musiman dalam kejadian gempa di Indonesia?**  
   → Dijawab dengan **heatmap waktu** yang memperlihatkan frekuensi per bulan dan tahun.

3. **Wilayah mana yang paling sering mengalami gempa dengan magnitudo besar (misal ≥ 5.0)?**  
   → Dijawab dengan peta interaktif + filter magnitudo.

4. **Bagaimana distribusi kedalaman gempa di berbagai wilayah Indonesia?**  
   → Dijawab dengan simbol/warna pada peta berdasarkan kategori kedalaman (shallow, intermediate, deep).

5. **Apakah terdapat tren peningkatan atau penurunan jumlah gempa dalam beberapa tahun terakhir?**  
   → Dijawab melalui heatmap atau grafik tren jumlah gempa per tahun.

6. **Bulan apa yang paling sering terjadi gempa bumi selama periode pengamatan?**  
   → Dijawab dengan heatmap frekuensi bulanan.

7. **Apakah terdapat hubungan antara magnitudo dan kedalaman gempa di wilayah tertentu?**  
   → Bisa dijawab dengan visual scatter plot interaktif tambahan atau informasi pada tooltip peta.

---